In [1]:
%load_ext autoreload
%autoreload 2
import os
import cv2
from vehicle_reid.embedder import Embedder
from vehicle_reid.evaluate import Evaluator
from vehicle_reid import exp_config
from vehicle_reid.common import *
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import h5py
import csv
EXP_ROOT = "/home/hthieu/AICityChallenge2019/track2_experiments/"
# EXP_ID = "180419_triplet-reid_pre-trained_densenet161_track2_small_512/"
EXP_ID = "270419_triplet-reid_pre-trained_densenet161_track2_small"
GPU_ID = "1"
EXP_DIR = os.path.join(EXP_ROOT,EXP_ID)
VIEW_QUE = os.path.join(EXP_DIR, "que_views.h5")

In [14]:
embedder = Embedder(EXP_DIR, GPU_ID)
config = exp_config.Test()
config.QUE_EMB_FILE = osp.join(EXP_DIR, config.QUE_EMB_FILE)
config.GAL_EMB_FILE = osp.join(EXP_DIR, config.GAL_EMB_FILE)
# config.FLIP = True
# config.CROP = 'five'
embedder.do_experiment_with_config(config)

Loading args from /home/hthieu/AICityChallenge2019/track2_experiments/270419_triplet-reid_pre-trained_densenet161_track2_small/args.json.
False False
INFO:tensorflow:Scale of 0 disables regularizer.
Save h5 file to:  /home/hthieu/AICityChallenge2019/track2_experiments/270419_triplet-reid_pre-trained_densenet161_track2_small/track2_query_embedding.h5
Restoring from checkpoint: /home/hthieu/AICityChallenge2019/track2_experiments/270419_triplet-reid_pre-trained_densenet161_track2_small/checkpoint-12729
INFO:tensorflow:Restoring parameters from /home/hthieu/AICityChallenge2019/track2_experiments/270419_triplet-reid_pre-trained_densenet161_track2_small/checkpoint-12729
Embedded batch 1024-1052/1052
Done with embedding, aggregating augmentations...
(1052, 512)
(1052, 512)
False False
INFO:tensorflow:Scale of 0 disables regularizer.
Save h5 file to:  /home/hthieu/AICityChallenge2019/track2_experiments/270419_triplet-reid_pre-trained_densenet161_track2_small/track2_test_embedding.h5
Restoring 

In [20]:
query_bus, test_bus = read_bus_query_test()
test_bus_train = load_h5("results_dists/testfulltrainbus798x5.h5")
query_to_test_track = load_h5("results_dists/querytestfull1052x798.h5")
exec_query_bus(test_bus_train,
               query_to_test_track,
               test_bus,query_bus)

[array([175, 536, 591, 161, 692, 565, 734, 569]),
 array([734, 591, 565, 692, 175, 536, 161, 569]),
 array([175, 692, 591, 161, 734, 536, 565, 569]),
 array([161, 536, 565, 734, 175, 591, 692, 569]),
 array([553, 617])]

In [2]:
config = exp_config.Test()
evaluator = Evaluator(EXP_DIR, GPU_ID)
evaluator.do_evaluate_with_config(config, query_extention = True)

QUE:  data/track2_query.csv
GAL:  data/track2_test_v3.csv
Load:  data/track2_query.csv
Load:  data/track2_test_v3.csv
Load:  /home/hthieu/AICityChallenge2019/track2_experiments/270419_triplet-reid_pre-trained_densenet161_track2_small/track2_query_embedding.h5
Load:  /home/hthieu/AICityChallenge2019/track2_experiments/270419_triplet-reid_pre-trained_densenet161_track2_small/track2_test_embedding.h5
(18290, 512)
(1052, 512)
Total queries:  1052
Results folder:  /home/hthieu/AICityChallenge2019/test_results/
Submission file:  track2.txt
Calculating batch 1024-1052/1052
mAP: 100.00% | top-1: 100.00% top-2: 100.00% | top-5: 100.00% | top-10: 100.00%


In [ ]:
config = exp_config.CustomEmbed()
view_classifier = Classifier(exp_config.ViewClassifier(), GPU_ID) 
view_classifier.classifiy(config.QUE_IMG_ROOT, config.QUE_FILE, VIEW_QUE)

In [6]:
import h5py
emb_mat = []
with open("/home/bnminh/projects/ai2/SOURCE/triplet-reid-pytorch-ai2/val/embds.csv") as fi:
    csv_reader = csv.reader(fi, delimiter = ',')
    for info in csv_reader:
        emb_mat.append(info)
emb_mat = np.array(emb_mat).astype(np.float32)
with h5py.File("bnminh_validate.h5", 'w') as f_out:
    f_out.create_dataset('emb', data=emb_mat)
    f_out.close()

In [8]:
class BNMinhValidate:
    def __init__(self):
        self.QUE_EMB_FILE="bnminh_validate.h5"
        self.GAL_EMB_FILE="bnminh_validate.h5"
        self.QUE_FILE="data/track2_validate.csv"
        self.GAL_FILE="data/track2_validate_v3.csv"
        self.QUE_IMG_ROOT="/home/hthieu/AICityChallenge2019/data/Track2Data/image_train/"
        self.GAL_IMG_ROOT="/home/hthieu/AICityChallenge2019/data/Track2Data/image_train/"
        self.RESULTS_ROOT="/home/hthieu/AICityChallenge2019/val_results/"
        self.QUE_VIEW_POINT="val_que_view_point.h5"
        self.GAL_VIEW_POINT="val_view_point.h5"
        self.FLIP = False
        self.CROP = False

In [21]:
class HTHieuValidate:
    def __init__(self):
        self.QUE_EMB_FILE="track2_validate_embedding.h5"
        self.GAL_EMB_FILE="track2_validate_embedding.h5"
        self.QUE_FILE="data/track2_validate.csv"
        self.GAL_FILE="data/track2_validate_v3.csv"
        self.QUE_IMG_ROOT="/home/hthieu/AICityChallenge2019/data/Track2Data/image_train/"
        self.GAL_IMG_ROOT="/home/hthieu/AICityChallenge2019/data/Track2Data/image_train/"
        self.RESULTS_ROOT="/home/hthieu/AICityChallenge2019/val_results/"
        self.QUE_VIEW_POINT="val_que_view_point.h5"
        self.GAL_VIEW_POINT="val_view_point.h5"
        self.FLIP = False
        self.CROP = False

In [14]:
evaluator = Evaluator("", GPU_ID)
config = BNMinhValidate()
evaluator.do_evaluate_with_config(config, query_extention = False)

QUE:  data/track2_validate.csv
GAL:  data/track2_validate_v3.csv
Load:  data/track2_validate.csv
Load:  data/track2_validate_v3.csv
Load:  bnminh_validate.h5
Load:  bnminh_validate.h5
(9155, 128)
(9155, 128)
(9155,)
(9155,)
Total queries:  9155
Results folder:  /home/hthieu/AICityChallenge2019/val_results/
Submission file:  track2.txt


ResourceExhaustedError: OOM when allocating tensor with shape[128,9155,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node cdist_3/sub (defined at /home/hthieu/AICityChallenge2019/triplet-reid/loss.py:20)  = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](cdist_3/ExpandDims, cdist_3/ExpandDims_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'cdist_3/sub', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hthieu/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hthieu/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instanc

In [ ]:
img = plt.imread('/home/hthieu/AICityChallenge2019/data/Track2Data/image_test/000290.jpg')
img2= plt.imread('/home/hthieu/AICityChallenge2019/data/Track2Data/image_test/011478.jpg')
img = cv2.resize(img, (256,128))
img2= cv2.resize(img2,(128,128))
img_croped = img[:,:128,:]
img = cv2.resize(img, (128,128))
plt.imshow(img_croped)
plt.figure()
plt.imshow(img2)
plt.figure()

img = img.astype(np.float32)
img2= img2.astype(np.float32)
img_croped = img_croped.astype(np.float32)

In [ ]:
images = [img, img_croped, img2]
images = [np.expand_dims(img, 0) for img in images]
images = np.concatenate(images, axis = 0)

In [ ]:
embedder = Embedder(EXP_DIR, GPU_ID)
embs = embedder.embed_images(tf.convert_to_tensor(images))

In [ ]:
embs.shape

In [ ]:
#img vs img 2
print(np.linalg.norm(embs[0]-embs[2]))
#img croped vs img 2
print(np.linalg.norm(embs[1]-embs[2]))